In [82]:
from pyspark.sql import SparkSession

app_name = "Absence Analysis"

spark = SparkSession.builder.master("local[*]").appName(app_name).getOrCreate()

In [83]:
# Load the data
df = spark.read.csv('../data/raw/Absence_3term201819_nat_reg_la_sch.csv', header=True, inferSchema=True)
df = df.sample(fraction=1.0)
df.printSchema()

root
 |-- time_identifier: string (nullable = true)
 |-- year_breakdown: string (nullable = true)
 |-- time_period: integer (nullable = true)
 |-- geographic_level: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- region_code: string (nullable = true)
 |-- region_name: string (nullable = true)
 |-- old_la_code: integer (nullable = true)
 |-- new_la_code: string (nullable = true)
 |-- la_name: string (nullable = true)
 |-- estab: integer (nullable = true)
 |-- laestab: integer (nullable = true)
 |-- urn: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- academy_type: string (nullable = true)
 |-- academy_open_date: string (nullable = true)
 |-- all_through: string (nullable = true)
 |-- num_schools: integer (nullable = true)
 |-- enrolments: integer (nullable = true)
 |-- sess_possible: long (nullable = true)
 |-- sess_overall: integer (nullable = true)
 |-- sess_authorised: integer (nullable = t

In [95]:
# data cleaning
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

df_cat = [col.name for col in df.schema.fields if isinstance(col.dataType, StringType)]
df_num = [col.name for col in df.schema.fields if col.name not in df_cat]
df = df.fillna('MISSING', subset=df_cat)
df = df.fillna(-1, subset=df_num)

# Replace ":" with "-1" first
df = df.replace(":", "-1", subset="sess_auth_ext_holiday")

df = df.withColumn("sess_auth_ext_holiday", col("sess_auth_ext_holiday").cast(IntegerType()))
print("Data type of sess_auth_ext_holiday:", df.schema["sess_auth_ext_holiday"].dataType)
df.select("sess_auth_ext_holiday").distinct().show(5)

df.limit(10).show()

Data type of sess_auth_ext_holiday: IntegerType()
+---------------------+
|sess_auth_ext_holiday|
+---------------------+
|                   -1|
|                  463|
|                  148|
|                  737|
|                   31|
+---------------------+
only showing top 5 rows

+---------------+--------------+-----------+----------------+------------+------------+-----------+-----------+-----------+-----------+-------+-----+-------+-------+--------------------+------------+-----------------+-----------+-----------+----------+-------------+------------+---------------+-----------------+--------------------+-----------------------+-------------------------+----------------------+------------------------------+-------------------------+------------------------+---------------------------+-----------------------------+--------------------------------+-----------------------------------+-------------------------------------+-----------------+----------------------+--------------

In [85]:
# first query from practical
# local authority, time period, sum of enrolments
p1_cols = ['la_name', 'time_period', 'enrolments']
list_of_las = df.select("la_name").distinct().collect()
print("List of Local Authorities:")
# for la in list_of_las[:10]:
#     print(la.la_name)

df.filter(df.la_name.isin([la.la_name for la in list_of_las[:3]]))\
    .filter(df.time_period > 201617)\
        .groupBy('la_name', 'time_period').sum('enrolments').show()


List of Local Authorities:
+--------------+-----------+---------------+
|       la_name|time_period|sum(enrolments)|
+--------------+-----------+---------------+
|North Tyneside|     201819|          77112|
|   Oxfordshire|     201819|         245589|
|        Bolton|     201819|         132495|
|North Tyneside|     201718|          75786|
|   Oxfordshire|     201718|         239703|
|        Bolton|     201718|         133002|
+--------------+-----------+---------------+



In [86]:
# school type
p1_cols = ['school_type', 'time_period', 'sess_authorised']
list_of_school_types = df.select("school_type").distinct().collect()
print("List of School Types:")
# for school_type in list_of_school_types:
#     print(school_type.school_type)

df.filter(df.school_type.isin([school_type.school_type for school_type in list_of_school_types]))\
    .filter(df.time_period < 200708)\
        .groupBy(df.school_type, df.time_period).sum('sess_authorised').show()

List of School Types:
+--------------------+-----------+--------------------+
|         school_type|time_period|sum(sess_authorised)|
+--------------------+-----------+--------------------+
|State-funded primary|     200607|           189763420|
|               Total|     200607|           320136897|
|             Special|     200607|             7380096|
|State-funded seco...|     200607|           229705680|
+--------------------+-----------+--------------------+



In [87]:
# all unauthorised absences
p1_cols = ['region_name', 'time_period', 'sess_unauthorised']
list_of_regions = df.select("region_name").distinct().collect()

df.filter(df.region_name.isin([region.region_name for region in list_of_regions]))\
    .filter(df.time_period > 201617)\
        .groupBy('region_name', 'time_period').sum('sess_unauthorised').show()

+--------------------+-----------+----------------------+
|         region_name|time_period|sum(sess_unauthorised)|
+--------------------+-----------+----------------------+
|          South West|     201819|              14394315|
|          South East|     201819|              25125525|
|             MISSING|     201819|              71273050|
|       East Midlands|     201819|              14876465|
|          North West|     201819|              27072735|
|          North East|     201819|              10331860|
|Yorkshire and the...|     201819|              22640265|
|     East of England|     201819|              17721435|
|       West Midlands|     201819|              20569760|
|        Inner London|     201819|               9169250|
|        Outer London|     201819|              16281015|
|          North West|     201718|              25105800|
|Yorkshire and the...|     201718|              21116965|
|       West Midlands|     201718|              19792550|
|             